In [ ]:
import os
import pickle
from typing import Tuple

import numpy
import numpy as np
import pandas
import pandas as pd
import tensorflow as tf
import tqdm
from keras.layers import TextVectorization
from numba import cuda
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from configuration import FilePaths
device = cuda.get_current_device()
device.reset()
from configuration import Config

In [ ]:

FILE_PATHS = FilePaths()

In [ ]:
# Set seeds to ensure reproduceability
tf.random.set_seed(Config.seed)
np.random.seed(Config.seed)
os.environ["PYTHONHASHSEED"] = str(Config.seed)

In [ ]:
# Read all three CSVs into a pandas data frame (I was provided with a combined_documents.csv, secondary_patient_level_annotations.csv and a secondary_problem_level_annotations.csv)
for i in tqdm(range(0, 3), ncols=100, desc="Loading data.."):
    raw_df_as_read = pd.read_csv(FILE_PATHS.RAW_DATA)
    patient_level_annotations_as_read = pd.read_csv(FILE_PATHS.PATIENT_LEVEL_DATA)
    problem_level_annotations_as_read = pd.read_csv(FILE_PATHS.PROBLEM_LEVEL_DATA)
print("------Loading is completed ------")

In [ ]:
# We are given lots of extraeneous information, so lets strip non relavent data. Note that patient ID is unique to each patient. 
# Firstly we create a copy of the data, create an additional column called Date which contains the date record was made
# Then we drop unnessicary columns, and filter all records by the study date (1/1/2019 -> 31/12/2019)

raw_df = raw_df_as_read.copy()

# Then we split the recorded date to DateTimeDay column, so we can get concat records existing on a single day"
raw_df[["DateTimeDay", "DateTimeSeconds"]] = raw_df["RecordedDate"].str.split(
    "T", expand=True
)
raw_df["Date"] = pd.to_datetime(raw_df["DateTimeDay"], format="%Y-%m-%d")


raw_df.drop(Config.raw_data_columns_drop, axis=1, inplace=True, errors='ignore') #Drop un-needed columns
raw_df = raw_df[(raw_df["Date"] >= Config.study_start_date) & (raw_df["Date"] <= Config.study_end_date)] #Only include patients withint study start date
raw_df

In [ ]:
patient_level_annotations = patient_level_annotations_as_read.copy()

patient_level_annotations.drop(Config.patient_level_columns_drop, axis=1, inplace=True)
patient_level_annotations

In [ ]:
study_patients = set(patient_level_annotations.loc[patient_level_annotations['Is this patient included in the study_1'] == 'Yes', 'PatientID'].tolist())

In [ ]:
problem_level_annotations = problem_level_annotations_as_read.copy()

# Drop all patients NOT in study
problem_level_annotations  =  problem_level_annotations[problem_level_annotations.PatientID.isin(study_patients)]

problem_level_annotations[["DateTimeDay", "DateTimeSeconds"]] = problem_level_annotations["DocumentDate"].str.split(
    " ", expand=True
)
problem_level_annotations["Date"] = pd.to_datetime(problem_level_annotations["DateTimeDay"], format="%d/%m/%Y")

problem_level_annotations.drop(Config.problem_level_columns_drop, axis=1, inplace=True)

In [ ]:
def check_multiple_entries(df)->list:
    "Check there are not multiple entries per day, if so remove these days from the data pool "
    multiple_entries_list = []
    for i in set(df.PatientID.tolist()):
        results = df.loc[df.PatientID ==i]
        if not results.Date.is_unique:
            multiple_entries_list.append(i)
    return multiple_entries_list

df = problem_level_annotations[~problem_level_annotations.PatientID.isin(check_multiple_entries(problem_level_annotations))]

if not len(check_multiple_entries(df)) == 0:
    raise Exception("Sorry, but it appears you have multiple categorised days!")
else:
    print("Data looks good")

In [ ]:

print(f"You have {len(df.PatientID.unique())} patients included in this study")
print(f"and a total of {len(df.index)} 24 hour periods classified")

In [ ]:
#Merge the EHRs to the DF
ehrs = []
for index, row in df.iterrows():
    ehr = raw_df[(raw_df.PatientId ==row.PatientID) & (raw_df.Date==row.Date)]
    ehr_list = ehr.Document.to_list()
    s = ' '.join(ehr_list)
    ehrs.append(s)
    
df.loc[:,'ehr'] = ehrs.copy()

In [ ]:
# Filter only the patient visits
df_patients_visits = df[df[Config.is_visit_column] == 'Yes'].copy()
patient_id_set:set = set(df_patients_visits[Config.patient_id_column].values)

tmp_list: list = []

print(f"{len(patient_id_set)} patients included in this study who had a visit")

In [ ]:
# TODO: Add a day of the year column in for further formatting. 

for patient_id in patient_id_set:
    patient_stream: list = []
    print(patient_id)
    print(df_patients_visits.loc[df_patients_visits[Config.patient_id_column] == patient_id])
    break

In [ ]:
# df_patients_visits['PatientID' == 977030]
df_patein

In [ ]:
df_patients_visits